# Better Way 38. 스레드에서 데이터 경쟁을 막으려면 Lock을 사용하자

전역 인터프리터 잠금 (Better Way 37) 배우고 나면, "코드"에서 상호배제잠금 (뮤텍스) 사용하지 않아도 된다?
- 파이썬 스레드가 여러 CPU 코어에서 병렬로 실행되는 것을 GIL이 이미 막았다면 프로그램의 자료 구조에 잠금이 설정되었을 것이다.
- 그러나 실제로는 그렇지 않다.
- 파이썬 스레드가 한 번에 하나만 실행되지만, 파이썬 인터프리터에서 자료 구조를 다루는 스레드 연산은 두 바이트코드 명령어 사이에서 인터럽트 될 수 있다.
- 따라서 여러 스레드에서 동시에 같은 객체에 접근한다면, 자료 구조의 불변성이 인터럽드 때문에 깨질 수도 있고, 프로그램은 오류로 남는다.

In [1]:
#예시: 전체 센서 네트워크에서 병렬로 여러 대상을 카운트하는 프로그램
class Counter(object):
    def __init__(self):
        self.count = 0
        
    def increment(self, offset):
        self.count += 1

In [2]:
#센서에서 읽는 작업에는 블로킹 I/O 필요--> 각 센서 별로 고유한 작업 스레드 있음
def worker(sensor_index, how_many, counter):
    for _ in range(how_many):
        # read from sensor
        counter.increment(1)

In [5]:
#센서별로 작업 스레드를 시작하고 읽기를 모두 마칠 때까지 기다림
from threading import Thread

def run_threads(func, how_many, counter):
    threads = []
    for i in range(5):
        args = (i, how_many, counter)
        thread = Thread(target=func, args=args)
        threads.append(thread)
        thread.start()
        
    for thread in threads:
        thread.join()

In [6]:
how_many = 10**5
counter = Counter()
run_threads(worker, how_many, counter)
print('Counter should be %d, found %d' % (5*how_many, counter.count))

Counter should be 500000, found 331460


파이썬 인터프리터 스레드가 한 번에 단 하나만 동작하는데도 위와 같은 오류가 발생하는 이유
- 파이썬 인터프리터는 모든 스레드가 거의 동등한 처리 시간 동안 실행하게 하려고, 실행 중인 모든 스레드 사이에서 공평성을 유지한다.
- 이를 위해 실행 중인 스레드를 잠시 중지하고 차례로 다른 스레드를 재개한다.
- 문제는 파이썬이 스레드를 정확히 언제 중제할지 모른다는 점이다.
- 스레드는 심지어 원자적 연산 (atomic operation)으로 보이는 작업 중간에서 멈출 수도 있다.

In [ ]:
counter.count += offset #이 메서드는 간단해 보인다.
##################################################
value = getattr(counter, 'count')
result = value + offset
setattr(counter, 'count', result) #+= 연산은 이와 같이 연산 3개를 수행하게 되고, 파이썬 스레드는 이 중간에도 중지될 수 있다.

In [ ]:
#예시: 두 스레드 사이의 안 좋은 상호 작용
value_a = getattr(counter, 'count') #스레드 A에서 실행
value_b = getattr(counter, 'count') #스레드 B로 컨텍스트 전환
result_b = value_b + 1
setattr(counter, 'count', result_b)
result_a = value_a + 1 #스레드 A로 다시 전환
setattr(counter, 'count', result_a)
#결과: 스레드 A는 스레드 B에서 카운터 증가를 실행하는 모든 작업을 없애버린다. (위의 밝기 예제와 동일함)

파이썬은 이와 같은 데이터 경쟁 (race)을 비롯한 다른 방식의 자료 구조 오염을 막기 위해 threading 내장 모듈에 여러 도구들을 갖추고 있다.
- 가장 간단하고 유용한 도구는 Lock 클래스 (상호배제잠금 기능 제공)
- 잠금을 이용하면: 여러 스레드가 동시에 접근하더라도 Counter 클래스의 현재 값을 보호할 수 있다. 한 번에 한 스레드만 잠금을 얻을 수 있기 때문이다.

In [7]:
from threading import Lock

class LockingCounter(object):
    def __init__(self):
        self.lock = Lock()
        self.count = 0
        
    def increment(self, offset):
        with self.lock:
            self.count += offset

In [8]:
counter = LockingCounter()
run_threads(worker, how_many, counter)
print('Counter should be %d, found %d' % (5*how_many, counter.count))

Counter should be 500000, found 500000


핵심 정리
- 파이썬에 전역 인터프리터 잠금이 있다고 해도 프로그램 안에서 실행되는 스레드 간의 데이터 경쟁으로부터 보호할 책임은 프로그래머에게 있다.
- 여러 스레드가 잠금 없이 같은 객체를 수정하면 프로그램의 자료 구조가 오염된다.
- 내장 모듈 threading의 Lock 클래스는 파이썬의 표준 상호 배제 잠금 구현이다.